In [12]:
import pandas as pd
import os

In [14]:
# Make Parquet CSV
folder_path = "../data/important-parquet/"
files = os.listdir(folder_path)
output_folder = "../data/important/"  # Specify the folder where you want to save the CSV files

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

for f in files:
    if f.endswith(".parquet"):
        # Read the Parquet file
        df = pd.read_parquet(os.path.join(folder_path, f))

        # Create the corresponding CSV file name
        csv_filename = os.path.splitext(f)[0] + ".csv"
        csv_path = os.path.join(output_folder, csv_filename)

        # Check if the CSV file already exists and remove it
        if os.path.exists(csv_path):
            os.remove(csv_path)
        
        # Save the Parquet data as a CSV file
        df.to_csv(csv_path, index=False)